# Intro

## CNN Architecture

- Conv layer - 6 filters, 3x3 size, stride 1
- Pooling layer - 2x2 size, stride 2
- Conv layer - 16 filters, 3x3 size, stride 1
- Pooling layer - 2x2 size, stride 2
- Fully connected - 46,656* > 120 > 84 > 2
*See calculation below

In [11]:
# Calculating fully connected layers input
c1 = output_size(224,3,0,1)
p1 = output_size(c1,2,0,2)
c2 = output_size(p1,3,0,1)
p2 = output_size(c2,2,0,2)

print(f'C1: {c1} > P1: {p1} > C2: {c2} > P2: {p2} = {int(p2)} pixels per side')
print(f'Fully connected layers inputs = {int(p2)**2*16:,} > {int(p2)} pixels squared x number of channels')

C1: 222.0 > P1: 111.0 > C2: 109.0 > P2: 54.5 = 54 pixels per side
Fully connected layers inputs = 46,656 > 54 pixels squared x number of channels


# Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models # add models to the list
from torchvision.utils import make_grid
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def output_size(w,k,p,s):
    '''
    Calculates the output size of a convolution/pooling layer.
    - w [int]: input volume
    - k [int]: Kernel size
    - p [int]: padding
    - s [int]: stride
    '''
    return ((w-k+2*p)/s)+1

# Transforms

We will be creating two transform objects, one for train data, and one for test data. 

In [12]:
train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

Images do not need to be augmented in the test set. 

In [13]:
test_transfrom = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

# Data sets and loaders

Loading the data can be done with the `datasets.ImageFolder` class.

In [16]:
root = '../Data/CATS_DOGS/'

train_data = datasets.ImageFolder(os.path.join(root,'train'),transform=train_transform)
test_data = datasets.ImageFolder(os.path.join(root,'test'),transform=train_transform)

torch.manual_seed(42)

train_loader = DataLoader(train_data,batch_size=10,shuffle=True,num_workers=3)
test_loader = DataLoader(test_data,batch_size=10,shuffle=True)

class_names = train_data.classes

print(class_names)
print(f'Training images available: {len(train_data):,}')
print(f'Testing images available:  {len(test_data):,}')

['CAT', 'DOG']
Training images available: 18,743
Testing images available:  6,251


# Reviewing data

In [24]:
# Grab the first batch of images
for images, labels in train_loader:
    break

# Print labels and classes
print(f'labels')
print()

tensor([0, 0, 1, 1, 1, 0, 1, 0, 1, 0])
